# Stock Sentiment Analysis Project

## Gathering the Article Data for our Stocks
Utilizing the News API service, we gather articles referencing the Stock Symbols that we are interested in. We take the JSON data returned, clean it, and store into our sqlite database for future reference.

In [ ]:
import sys
from news_finders import content_scraper
from news_finders import newsApi
from utils import file_utils
from utils import sqlite_utils
from datetime import datetime, timedelta
  

STOCKS = ["AAPL", "ABBV", "TXN"]


date= datetime.today() - timedelta(days=28)

reformatedDate= str(date.month) + "-" + str(date.day) + "-" + str(date.year)


articles = []
articles = newsApi.getArticlesForMultipleStocks(STOCKS, reformatedDate)



conn= sqlite_utils._createConnection(r"/var/stockSA/stockSentiment.db")

if conn is None: 
    print("Failed to open database connection")
    sys.exit(1)

sqlite_utils.insertStockArticles(conn, articles)
print(sqlite_utils._findStockArticlesForSymbol(conn, "AAPL")[0])

## Gathering the Historical Price Data for our Stocks
Utilizing the Yahoo Financials API, we gather stock finance data per each company that we are interested in and store that data in a Stock Pricing Database. 

In [ ]:
import sys
from yahoofinancials import YahooFinancials
from utils import file_utils
from utils import sqlite_utils
from datetime import datetime, timedelta


# connecting to the database
conn = sqlite_utils._createConnection(r"/var/stockSA/stockPricing.db")
if conn is None:
    print("Failed to open database connection")
    sys.exit(1)
sqlite_utils._createStockPricingTable(conn)

while True :
    ticker = input('Enter a stock symbol or to quit program, type quit: ')
    if ticker == 'quit' :
        quit()

    try: 
        yahoo_financials = YahooFinancials(ticker) 
        # call to the YahooFinance tool function
        historicalStockPrices = yahoo_financials.get_historical_price_data(
                "2020-06-09", "2020-07-08", "daily")
        
        # creating our variable prices, storing all of the data in a list
        if historicalStockPrices[ticker] is None:
            print('Failed to get stock prices')
            continue
        
        prices = historicalStockPrices[ticker]['prices']
        sqlite_utils.insertPrices(conn, ticker, prices)	

    except Exception as e: 
    	print("Invalid StockSymbol, no data ", e)
    	continue
    
    print(prices)

### Pulling Data from stockSentiment database

In [3]:
import sys
import pandas as pd
from utils import file_utils
from utils import sqlite_utils


conn= sqlite_utils._createConnection(r"/var/stockSA/stockSentiment.db")

if conn is None: 
    print("Failed to open database connection")
    sys.exit(1)

#print(sqlite_utils._findStockArticlesForSymbol(conn, "AAPL")[0])
    
#print(sqlite_utils._findAllStockArticlesBetweenDates(conn, "2020-03-09", "2020-11-14", "AAPL"))

dataFrame = pd.read_sql_query('SELECT * FROM stockArticles WHERE date >= "2020-03-16" AND date <= "2020-11-14" AND stockSymbol="AAPL" ORDER BY date DESC', conn)
dataFrame

,stockSymbol,name,url,content,description,scraper,date,sentiment
0,AAPL,Peter Lynch disciple William Danoff manages ov...,https://www.businessinsider.com/investing-tips...,"When William Danoff, the portfolio manager of ...","Summary List Placement\n\nWhen William Danoff,...",NewsAPI,2020-11-13,None
1,AAPL,Need to Know: Goldman Sachs has just boosted i...,https://www.marketwatch.com/story/goldman-sach...,The news that the COVID-19 vaccine from drugma...,A look at why Goldman Sachs has increased its ...,NewsAPI,2020-11-11,None
2,AAPL,Stock Market Today With Jim Cramer: Apple Even...,https://www.thestreet.com/investing/stock-mark...,Stocks were mixed Tuesday following a Wall Str...,Jim Cramer discusses the latest stock market n...,NewsAPI,2020-11-10,None
3,AAPL,Stock market news live updates: Stock futures ...,https://ca.news.yahoo.com/election-2020-stock-...,Stocks were mixed Tuesday as investors reined ...,<ol><li>Stock market news live updates: Stock ...,NewsAPI,2020-11-10,None
4,AAPL,: A strong year-end rally lies ahead. It’s tim...,https://www.marketwatch.com/story/a-strong-yea...,U.S. stocks look set to continue their rally a...,U.S. stocks look set to continue their rally a...,NewsAPI,2020-11-09,None
5,AAPL,FAANG stands for five very successful tech com...,https://www.businessinsider.com/what-is-faang,"If you follow the financial or business news, ...",Summary List PlacementIf you follow the financ...,NewsAPI,2020-11-06,None
6,AAPL,The Ratings Game: Software stocks rocket highe...,https://www.marketwatch.com/story/software-sto...,Software stocks are surging in Wednesday tradi...,Software stocks are surging in Wednesday tradi...,NewsAPI,2020-11-04,None
7,AAPL,Apple's Still Getting Punished on Wall Street ...,https://gizmodo.com/apples-still-getting-punis...,Apples Q4 earnings report could have been an u...,Apple’s Q4 earnings report could have been an ...,NewsAPI,2020-11-02,None
8,AAPL,Earnings Watch: Companies try to avoid Electio...,https://www.marketwatch.com/story/companies-tr...,The week ahead brings another packed earnings ...,While Tuesday afternoon and Wednesday morning ...,NewsAPI,2020-11-01,None
9,AAPL,Europe Markets: European stocks crawl off five...,https://www.marketwatch.com/story/european-sto...,European stocks stabilized Thursday after endi...,European stocks stabilized Thursday after endi...,NewsAPI,2020-10-29,None
